# **OCR**

En este notebook veremos cómo **parsear documentos** con [LlamaParse](https://pypi.org/project/llama-parse/) para extraer texto de forma estructurada y confiable.  

El flujo principal será:

1. **Instalación de dependencias.**  
2. **Importación de librerías.**  
3. **Configuración previa (variables de entorno).**  
4. **Definir la ruta del documento PDF final.**  
5. **Establecer instrucciones de parseo (OCR y formato).**  
6. **Procesar el PDF con LlamaParse.**  
7. **Almacenar el texto resultante en un archivo local.**  
8. **Visualizar una sección del documento parseado.**  



## **Notas**
- **LlamaParse** solo procesa documentos pdf. Si necesitas extraer texto de un archivo que NO sea PDF, consulta con el desarrollador para acceder a la **librería de conversión** para transformarlo en un PDF válido.
- Configura la variable de entorno `LLAMA_CLOUD_API_KEY` con tu clave real para poder usar **LlamaParse**.
- Ajusta los parámetros de `LlamaParse` (por ejemplo, `premium_mode`, `language`, etc.) en función de tus necesidades de OCR y parseo.
- Para utilizar `premium_mode` consulta con el encargado de proporcionar las `API_KEY` debido a que su uso implica un costo adicional.



#  1. Instalación de dependencias


In [1]:
!pip install llama-parse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.3 MB/s eta 0:00:00


#2. Importar librerías

In [2]:
import os
import nest_asyncio
from llama_parse import LlamaParse

# 3. Configuración Previa


In [7]:
# Ajusta tu clave de LlamaParse aquí: (CONSULTA CON EL DESARROLLADOR PARA OBTENER LA API-KEY)
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"

# 4. Definir la ruta del documento PDF

Puedes encontrar algunos archivos de ejemplo en el la carpeta del repositorio

In [4]:
path_pdf_document = "/content/11050.pdf"

print(f"Usaremos el archivo PDF en: {path_pdf_document}")

Usaremos el archivo PDF en: /content/11050.pdf


# 5. Definición de las Instrucciones de Parseo

Consiste en explicar, en términos generales la estructura del documento que se proporcionará y lo que se espera que haga el parser (por ejemplo, recuperar integramente el contenido y en idioa español)


In [5]:
parsing_instructions = '''
The document contains structured text that includes headers, dates, names of individuals, institutions, and places, as well as numbered sections, lists, and tables. Many of these documents feature text that is highly deteriorated and requires careful interpretation, contextualization, and reconstruction. Non-essential elements, such as scratches, signatures, and diagonal annotations, must be omitted.
To ensure the integrity of the extracted information, the text must be preserved in its literal form without summarizing, paraphrasing, or modifying its meaning. Treat each document as evidentiary material, prioritizing rigorous and faithful extraction practices.
Recognize that most documents are typewritten, which introduces specific challenges such as ink smudges, duplicated letters, and words split across lines. Proactively correct these errors, ensuring clarity and precision in the recovered content.
While processing, handle page breaks to maintain the narrative flow, and retain the original structure of tables and lists without flattening their format. Extract and clearly highlight key names, dates, and places. Quotes, dialogues, abbreviations, and codes must be preserved exactly as they appear. Non-textual elements should be omitted to focus solely on the written content.
The output should adhere to Markdown formatting conventions but must not include code block tags such as markdown. Use bold formatting for headers, numbered or bulleted lists for structured sections, block quotes for quoted material, and Markdown-compatible tables for tabular data. Narrative text should be continuous, providing both factual information and detailed descriptions, while preserving the integrity and original context of the document. Additionally, prioritize correcting OCR-related errors caused by typewriter artifacts whenever possible.
The response must be exclusively in the original language of the document, which is generally Spanish. No translation or language modification is allowed.
'''

print("Instrucciones de parseo definidas.")


Instrucciones de parseo definidas.


# 6. Carga y Procesamiento del Documento con LlamaParse

Para un mayor detalle de la configuración, visita las [opciones del parser](https://https://docs.cloud.llamaindex.ai/llamaparse/features/parsing_options)

In [8]:
# Permite que asyncio funcione sin problemas en entornos con un loop de eventos en ejecución (como Jupyter o Colab)
nest_asyncio.apply()

# Carga y procesa el documento PDF usando LlamaParse con las configuraciones especificadas
document = LlamaParse(
    result_type="markdown",        # Define que el resultado será en formato Markdown
    premium_mode=False,            # No usa el modo premium (costo extra)
    parsing_instructions=parsing_instructions,  # Instrucciones personalizadas para el parsing
    language='es',                 # Define el idioma español para la extracción de texto. Un listado extenso de lenguajes admitidos está disponible en https://github.com/run-llama/llama_cloud_services/blob/main/llama_cloud_services/parse/utils.py#L16
    skip_diagonal_text=True,        # Omite texto en diagonal para mejorar la limpieza del documento
    do_not_unroll_columns=False     # LlamaParse intentará desenrollar las columnas (colocándolas una después de la otra en el orden de lectura). Si se establece do_not_unroll_columns = True, LlamaParse no podrá hacerlo.
).load_data(path_pdf_document)      # Carga y extrae los datos del archivo PDF especificado


print("\n Documento parseado correctamente. \n")
print(f"Total de secciones extraídas: {len(document)}")


Started parsing the file under job_id 1903454b-ee51-455e-b127-53d2abf9081a
.Documento parseado correctamente.
Total de secciones extraídas: 16


# 7. Guardar el Texto Recuperado en un Archivo


In [9]:
full_text = ''
for i in range(len(document)):
    full_text += '\n\n'
    full_text += document[i].text

# Crear archivo de salida
path_txt_document = os.path.splitext(path_pdf_document)[0] + ".txt"

# Escribir el contenido parseado en un .txt
with open(path_txt_document, "w") as file:
    file.write(full_text)

print(f"El archivo parseado ha sido guardado en: {path_txt_document}")


El archivo parseado ha sido guardado en: /content/11050.txt


# 8. Visualización de una Página o Sección


In [10]:
# Puedes cambiar el índice en pagina_a_visualizar para ver otras partes del documento

pagina_a_visualizar = 3

if pagina_a_visualizar < len(document):
    print(document[pagina_a_visualizar].text)
else:
    print(f"Índice fuera de rango. El documento sólo tiene {len(document)} partes extraídas.")


# RESOLUCION EXENTA ELECTRONICA N° 11050

Santiago, 01 de Marzo de 2022

personas y, por tanto, la ejecución de tales estudios en los tiempos que habitualmente se requieren para estos efectos.

En respuesta a la solicitud indicada en el párrafo anterior, el SEA, mediante Resolución Exenta N°197 de fecha 28 de septiembre de 2020 otorgó a Grenergy la extensión del plazo solicitado hasta el 30 de noviembre de 2020, esto es, por un término de 2 meses adicionales al de la suspensión por dicha autoridad ya decretada, reconociendo con ello, los argumentos referidos a la fuerza mayor que produjeron las medidas, restricciones sanitarias y de movilidad tomadas por la autoridad durante la pandemia.

Luego, mediante carta de fecha 27 de noviembre de 2020, Grenergy solicitó al SEA que se mantuviera la suspensión del plazo de evaluación del procedimiento ambiental del Proyecto hasta el 15 de enero de 2021.

Cabe señalar que, en dicho momento, continuaban las restricciones ya señaladas.

En respuesta